In [ ]:
import sqlite3

In [ ]:
conn = sqlite3.connect('nominations.db')

### Check Contents of Nomations Table

In [ ]:
query = '''
PRAGMA table_info(nominations);
'''
schema = conn.execute(query).fetchall()
for row in schema:
    print(row)
query='''
SELECT *
FROM nominations
LIMIT 10;
'''
first_ten = conn.execute(query).fetchall()
for row in first_ten:
    print(row)

### Create and Fill Ceremonies Table

In [ ]:
years_hosts = [
    (2010, "Steve Martin"),
    (2009, "Hugh Jackman"),
    (2008, "Jon Stewart"),
    (2007, "Ellen DeGeneres"),
    (2006, "Jon Stewart"),
    (2005, "Chris Rock"),
    (2004, "Billy Crystal"),
    (2003, "Steve Martin"),
    (2002, "Whoopi Goldberg"),
    (2001, "Steve Martin"),
    (2000, "Billy Crystal")
]

In [ ]:
query = '''
CREATE TABLE ceremonies (
id integer PRIMARY KEY,
Year integer,
Host text
)
'''
conn.execute(query)

insert_query = '''
INSERT INTO ceremonies (Year, Host) VALUES (?,?);
'''
conn.executemany(insert_query, years_hosts)

### Verify Ceremonies Table Created and Properly Populated

In [ ]:
query = '''
PRAGMA table_info(ceremonies)
'''
table_info = conn.execute(query).fetchall()
for row in table_info:
    print(row)
query = '''
SELECT *
FROM ceremonies
LIMIT 10;
'''
first_ten = conn.execute(query).fetchall()
for row in first_ten:
    print(row)

In [ ]:
query = '''
PRAGMA foreign_keys = ON;
'''
conn.execute(query)

### Recreate Nominations Table to Include Ceremony Id

In [ ]:
query = '''
SELECT nominations.category, nominations.nominee,
nominations.movie, nominations.character, nominations.won,
ceremonies.id
FROM nominations
INNER JOIN ceremonies ON
nominations.year == ceremonies.year
'''

joined_nominations = conn.execute(query).fetchall()

In [ ]:
create_table = '''
CREATE TABLE nominations_two (
id integer PRIMARY KEY,
category text,
nominee text,
movie text,
character text,
won integer,
ceremony_id integer,
FOREIGN KEY(ceremony_id) REFERENCES ceremonies(id)
)
'''
conn.execute(create_table)

insert_query = '''
INSERT INTO nominations_two (category, nominee, movie, character, won, ceremony_id)
VALUES (?,?,?,?,?,?)
'''
conn.executemany(insert_query, joined_nominations)

In [ ]:
query = '''
SELECT *
FROM nominations_two
LIMIT 5;
'''
nominations_two = conn.execute(query).fetchall()
nominations_two

In [ ]:
drop_query = '''
DROP TABLE nominations
'''
name_change = '''
ALTER TABLE nominations_two
RENAME TO nominations
'''

conn.execute(drop_query)
conn.execute(name_change)

In [ ]:
query = '''
SELECT *
FROM nominations
LIMIT 10;
'''

print(conn.execute(query).fetchall())

### Create Movies, Actors and Movies_Actors Tables

In [ ]:
movie_table = '''
CREATE TABLE movies (
id integer PRIMARY KEY,
movie text
)
'''

actors_table = '''
CREATE TABLE actors (
id integer PRIMARY KEY,
actor text
)
'''

movie_actor_table = '''
CREATE TABLE movies_actors (
id integer PRIMARY KEY,
movie_id integer,
actor_id integer,
FOREIGN KEY(movie_id) REFERENCES movies(id),
FOREIGN KEY(actor_id) REFERENCES actors(id)
)
'''

In [ ]:
conn.execute(movie_table)
conn.execute(actors_table)
conn.execute(movie_actor_table)

### Populate Movies Table

In [ ]:
pop_movies = '''
INSERT INTO movies (movie)
SELECT DISTINCT movie
FROM nominations
'''
query = '''
SELECT *
FROM movies
LIMIT 10;
'''
conn.execute(pop_movies)
print(conn.execute(query).fetchall())

### Populate Actors Table

In [ ]:
pop_actors = '''
INSERT INTO actors (actor)
SELECT DISTINCT nominee
FROM nominations;
'''
query = '''
SELECT *
FROM actors
LIMIT 10;
'''

conn.execute(pop_actors)
print(conn.execute(query).fetchall())

### Populate Movies_Actors Table with INNER JOIN Query

In [ ]:
pop_actors_movies = '''
INSERT INTO movies_actors (movie_id, actor_id)
SELECT movies.id, actors.id FROM nominations
INNER JOIN movies ON nominations.movie == movies.movie
INNER JOIN actors ON nominations.nominee == actors.actor;
'''
query = '''
SELECT *
FROM movies_actors
LIMIT 10;
'''
conn.execute(pop_actors_movies)
print(conn.execute(query).fetchall())

In [ ]:
conn.close()